In [ ]:
#!pip install torch==1.6.0
!pip install transformers==3.3.1
!pip install seqeval
!pip install fastprogress
!pip install attrdict

해당 과제의 코드는 박장원님의 "https://github.com/monologg/KoELECTRA/tree/master/finetune" 레포지토리의 코드를 변경하여 만들어졌습니다.

추신. 좋은 코드 만들어주시고, 코드 사용을 흔쾌히 허락해주신 박장원님께 감사인사 드립니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/Colab\ Notebooks/WEEK_NLP_day20_assignment

학습을 시작하기 전에 같이 동봉된 "preprocessing.ipynb"를 실행하여 나오는 "train.tsv","dev.tsv"를 "data/custom-ner"에 넣어줍니다.

In [ ]:
!python3 run_ner.py --task custom-ner --config_file koelectra-small.json # config/custom-ner 에서 원하는 모델의 config를 변경하고 넣어주세요.

In [ ]:
from transformers import ElectraTokenizer, ElectraForTokenClassification  #다른 모델을 쓸 때는 tokenizer(ex.ElectraTokenizer)와 model(ex.ElectraForTokenClassification)를 바꿔주어야 합니다.

  
tokenizer = ElectraTokenizer.from_pretrained("")  #tokenizer를 바꾸었다면 from_pretrained에 들어가는 부분을 바꿔주어야 합니다. model config에 있는 model 주소를 적어주시면 됩니다. ex. monologg/koelectra-base-v3-discriminator

model = ElectraForTokenClassification.from_pretrained("checkpoint가 있는 디렉토리 기입")


In [ ]:
label_list=['DT-B','DT-I','LC-B','LC-I','OG-B','OG-I','PS-B','PS-I','QT-B','QT-I','TI-B','TI-I','O']

Kaggle Competition page에서 "test.txt"를 다운 받아서 "data/custom-ner"에 넣어줍니다.

In [ ]:
texts=[]
with open("data/custom-ner/test.txt") as f:
  for line in f:
    texts.append(line.rsplit("\n")[0])

In [ ]:
import torch
from tqdm.auto import tqdm

In [ ]:
labels=[]
for idx, sequence in tqdm(enumerate(texts)):
  tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
  inputs = tokenizer.encode(sequence, return_tensors="pt")
  outputs = model(inputs)
  predictions = torch.argmax(outputs[0], dim=2)
  labels_=[]
  for i,j in zip(tokens,predictions[0].tolist()):
    if i in ['[CLS]', '[SEP]']:
      continue
    if "##" in i:
      continue
    labels_.append(label_list[j])
  labels.extend(labels_)


"submission.csv"를 kaggle competition에 제출합니다.

데이터 전처리,모델 config 튜닝을 이용하여 더 좋은 성적을 얻어보세요.

In [ ]:
with open("submission.csv","w") as f:
  f.write("id,tag\n")
  for idx,tag in enumerate(labels):
    f.write(str(idx))
    f.write(",")
    f.write(tag)
    f.write("\n")